In [311]:
import requests
import pickle
import time
from bs4 import BeautifulSoup
import datetime

In [4]:
with open('../data/yelp_businesses.pickle', 'rb') as pickleReader:
    business_dict = pickle.load(pickleReader)

In [264]:
business_ids = [k for k, v in business_dict.items() if v['location']['state']=='AZ' ]

In [262]:
business_dict[business_ids[0]]['location']['state']=='AZ'

True

In [265]:
len(business_ids)

9193

In [30]:
business_dict[business_ids[0]]

{'id': 'JzOp695tclcNCNMuBl7oxA',
 'alias': 'four-peaks-brewing-tempe',
 'name': 'Four Peaks Brewing',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/pPSp8RkEjojUrQ4Gx3n_2w/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/four-peaks-brewing-tempe?adjust_creative=ihkhZPaVUfUYeK7PoCZp6w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ihkhZPaVUfUYeK7PoCZp6w',
 'review_count': 2119,
 'categories': [{'alias': 'burgers', 'title': 'Burgers'},
  {'alias': 'pizza', 'title': 'Pizza'},
  {'alias': 'sandwiches', 'title': 'Sandwiches'}],
 'rating': 4.5,
 'coordinates': {'latitude': 33.419568, 'longitude': -111.916097},
 'transactions': ['pickup', 'delivery'],
 'price': '$$',
 'location': {'address1': '1340 E 8th St',
  'address2': 'Ste 104',
  'address3': '',
  'city': 'Tempe',
  'zip_code': '85281',
  'country': 'US',
  'state': 'AZ',
  'display_address': ['1340 E 8th St', 'Ste 104', 'Tempe, AZ 85281']},
 'phone': '+18444394722',
 'display_phone': '(844) 

In [ ]:
reviews_dict = {}

In [10]:
url = business_dict[business_ids[0]]['url']
r = requests.get(url)
r.status_code
soup = BeautifulSoup(r.content, "lxml")

In [162]:
def get_review_ids(soup):
    '''
    Doc Stings
    Convert the request content into a string to extract the data-review-id on the yelp page.
    
    INPUT:  Beautiful soup lxml
    OUTPUT: list of data review ids from Yelp page
    '''
    soup_string = str(soup)
    data_review_ids = []
    x = soup_string.find('data-review-id')
    next_id=x
    while next_id > 0:

        y=x+soup_string[x:100+x].find('"')
        z=y+soup_string[y+1:100+y].find('"')

        data_review_id = soup_string[y+1:z+1]
        if data_review_id not in data_review_ids:
            data_review_ids.append(data_review_id)

        next_id = soup_string[x+1:].find('data-review-id')

        x += next_id+1 
        if x>1:
            x+=1
            
    return data_review_ids

In [184]:
def append_review(review_id_list, reviews_dict):
    '''
    Doc Stings
    Convert the request content into a string to extract the data-review-id on the yelp page.
    
    INPUT:
    OUTPUT:
    '''
    for review in review_id_list:
        div = soup.find("div", {"data-review-id": review})
        if div!=None:
            if div.find("p").text != '\n            Was this review …?\n    ':
                reviews_dict[review] = div.find("p").text

    return reviews_dict

In [186]:
reviews_dict = append_review(review_ids_list, reviews_dict)

In [187]:
for k, v in reviews_dict.items():
    print(k,'\n', v, '\n\n')

ZnuNh1OvHGQ9V37bWcYinw 
 This place can get busy. But go with the right company and grab a drink, the wait won't be bad.Good brewery menu and good beer. I mainly go for the ambience. The beer and food is just a bonus. 


cnpjTFbonpc-Ay_mQdYXzQ 
 This four peaks happens to be down the street from my work so i frequent here for lunch.  Four peaks not only has some of the better craft beers around by their food is always quite good.  Some of my favorites on the menu would include the chicken strips, fish and chips, the chicken enchiladas with the egg, and their brunch burger (if it's available)This place is always quite busy but they have built a great sized patio and inside area as well to accommodate the crowds.  Misters and large umbrellas makes for a nice place to sit even in the summer heat. The service staff I always friendly and you can't go wrong with some quality beer and a good meal! 


hdoB0MXOXbWlRKhI_nVYxQ 
 This place has gone down hill lately. This was the first time I had 

In [205]:
def pull_yelp_review(nbr_key, reviews_dict):
    if 'reviews_pulled' not in business_dict[business_ids[nbr_key]].keys():
        url = business_dict[business_ids[nbr_key]]['url']
        r = requests.get(url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.content, "lxml")
            review_ids_list = get_review_ids(soup)
            new_reviews_dict = append_review(review_ids_list, reviews_dict)
            business_dict[business_ids[nbr_key]]['reviews_pulled'] = True
            return new_reviews_dict, 'done'
        else:
            print('yelp page request error')
            return reviews_dict, 'error'
    else:
        print('review previously pulled.')
        return reviews_dict, 'previously'

In [248]:
reviews_dict, status = pull_yelp_review(4, reviews_dict)

(80, 'done')

In [ ]:
nbr_key = 0

In [371]:
nbr_key += 1

In [362]:
ds = open('../data/data_scraping.txt', 'a')

In [376]:
while nbr_key < 2300:
    if 'reviews_pulled' not in business_dict[business_ids[nbr_key]].keys():
        url = business_dict[business_ids[nbr_key]]['url']
        r = requests.get(url)

        print('Key:', nbr_key)
        print('Request Status:', r.status_code)
        if r.status_code == 200:
            time.sleep(45)
            soup = BeautifulSoup(r.content, "lxml")

            review_ids_list = get_review_ids(soup)

            reviews_dict = append_review(review_ids_list, reviews_dict)

            business_dict[business_ids[nbr_key]]['reviews_pulled'] = True
        
        if ds.closed:
            ds = open('../data/data_scraping.txt', 'a')
            print('opened data scraping log file.')
        ds.write("{date}\t{key}\t{status}\n".format(date=datetime.datetime.now(), key=nbr_key, status=r.status_code))
        
        print(business_ids[nbr_key] , 'Done', business_dict[business_ids[nbr_key]]['reviews_pulled'])
        print(len(reviews_dict), status)

    nbr_key += 1
    print('Next Key:', nbr_key)

ds.close()
with open('../data/yelp_review.pickle', 'wb') as pickleWriter:
    pickle.dump(reviews_dict, pickleWriter, protocol=2)

print('Batch Done at Key:', nbr_key)

Key: 2250
Request Status: 200
opened data scraping log file.
0gXm4mso1guiudIA71SPzQ Done True
14187 done
Next Key: 2251
Key: 2251
Request Status: 200
8JGo_-zs3tKjdBq6ew0Ehg Done True
14187 done
Next Key: 2252
Key: 2252
Request Status: 200
9wlGQfWA7OFoF-mqVkMDmQ Done True
14187 done
Next Key: 2253
Key: 2253
Request Status: 200
MtBEpp_txJp5OYoHGAKiAA Done True
14187 done
Next Key: 2254
Key: 2254
Request Status: 200
nOSXhpFwzojs-hLsOq9fLQ Done True
14187 done
Next Key: 2255
Key: 2255
Request Status: 200
BTmxoRoPPbWJA2O8XRKMAA Done True
14187 done
Next Key: 2256
Key: 2256
Request Status: 200
UdEmYOnk2iJDY9lpEPAlJQ Done True
14187 done
Next Key: 2257
Key: 2257
Request Status: 200
Jd2NUZeVeyWi1tTNKXMt1Q Done True
14187 done
Next Key: 2258
Key: 2258
Request Status: 200
sc4w3F0OVUOsyuLeZoJ_9g Done True
14187 done
Next Key: 2259
Key: 2259
Request Status: 200
tXR5EE1q4JesG0XWawXWUw Done True
14187 done
Next Key: 2260
Key: 2260
Request Status: 200
0C1ME-oi2GtsHr8xsV9fsA Done True
14187 done
Next 

In [377]:
print('Key:', nbr_key)

Key: 2300


In [365]:
ds.close()

In [363]:
ds.write("{date}\t{key}\t{status}\n".format(date=datetime.datetime.now(), key=nbr_key, status=r.status_code))

36

In [370]:
if ds.closed:
    ds = open('../data/data_scraping.txt', 'a')
    print('opened data scraping log file.')

In [404]:
url = business_dict[business_ids[nbr_key]]['url']
r = requests.get(url)

print('server status', r.status_code)

server status 200


In [405]:
soup = BeautifulSoup(r.content, "lxml")

review_ids_list = get_review_ids(soup)

In [330]:
reviews_dict = append_review(review_ids_list, reviews_dict)

business_dict[business_ids[nbr_key]]['reviews_pulled'] = True

In [ ]:
time.sleep(30)

In [282]:
print((len(business_ids)*30)/60/60, 'hours')
print((len(business_ids)*30)/60/60/24, 'days')

76.60833333333333 hours
3.192013888888889 days


In [338]:
with open('../data/yelp_review.pickle', 'wb') as pickleWriter:
    pickle.dump(reviews_dict, pickleWriter, protocol=2)

In [293]:
with open('../data/yelp_review.pickle', 'rb') as pickleReader:
    test = pickle.load(pickleReader)

In [394]:
for k in reviews_dict.keys():
    print(k)


ZnuNh1OvHGQ9V37bWcYinw
cnpjTFbonpc-Ay_mQdYXzQ
hdoB0MXOXbWlRKhI_nVYxQ
ChsGV5bZUmbeeqVT-k0nXg
ukff2zCrSz_MF-MrRukgeg
Ke3Dgn2x5vXxmYHngRxevw
AvW35g-FUx9LvVChKc57fg
SC66JtvFykAZWSb8ZTe-IQ
uW_3PW4QwoErvA_XaEaVCQ
UmAgPky1ng30vsTLxlrsuA
mQe3UJRZCOiz-xY4lODYxg
8jYsEl63Lm3ckUEvYkW4kQ
kL-ro-laj4I2siQrgeki8Q
pJizrSv-4IwsPo2pAzPmIA
AHG4DZ2GYjJXcg7LyvPMhQ
awK4THNcRBCguDKa8Q_5NA
JVITPvg5vCJ6AUeY8yQC6g
ncbH-qsrg7cBhle-DSNzbQ
ZtAPupGj0bLOaFe8hFd7dw
99end7AF6NiJsn2AUBRzdw
4cvSMVJnmtIyy4jRD66mFg
M8fv9I04e0J3L-UxK4xA4A
2jlf6NA5ZetNOO4jpVLWqg
wmEl8uZJH-JrkK5ZhXijkw
X3GPGOSCMLDWbpVRZG_AIA
pVQql9djXWWXO6NyZDf9HQ
z9jmAT4uV72TmMeAu0Ixng
K-hAlpV3od8dzjSEmd69UQ
aTR8H6gsmATcbpH6XWYICw
mrS2rW8qrajqfEnrJywt0A
KDp1_QwBKswsdFmM7cc2jw
PNcsxa_oVyMstOCUDbNFLQ
6Z1Ms5vzhSpbDoERL5JNLg
pY3jqrYZPSvq9OScilxBWQ
WuUIdgETLIgKrLLACQhwFQ
LM-2PQwuW_HQ2YtLM4g2nw
7NPKzfW0mFz6Egqx4ZITGA
FnlmMtrNHNcR0comVYYtKQ
r9XVUjLhkysR7TrjscSnwQ
HtGHcguWuAhpJGjmbKwYAQ
a74dfOZNoHxnvIIM6dHzwA
4ux9UIOJQIkLOoL3wzpHTw
2WvJ6Nc_mK2R7kJqSNjsWg
D5vYrSdaP6B

ZQFjinb6aQRzRMOF_yNAiQ
ZH5vwnAH3n1nRdC67MG4cg
pgZ7H0Ebr6ljCOy4wY6ONw
cignYY42fva3hVDcM1BikA
coFvPfMlZTIvSdxsLJLiIw
zXaxNUh9aZY8vpB-zey-QA
YX2B12AOBZc5HtBRVK68HQ
pO6feL0HFZ9n7ucLDuaqJA
7TnH3epwNXS33FlTL1qEVg
9xGKQR6pFMeo0A3mGcSyTg
PBrWxip1dGdsLQEU2FS3mg
3dcVwI-d_XVuqH2ebyZ27w
tBoe-LoFVsgH11S8vyjv3g
W1zD9pwv2py7iL0iq20NAw
sHoK87BlAGq_WB5MzhPNNg
v_PwOBQPn1ARn85WWj9i7w
5YaJ55JqEi0zk3mBfQPF7g
I2GVoGIjlmWaQOpyTSpf3A
IjvE3HEXLHauBxOQag2A8g
uTGyF1aSTR7VzQr8FYt23g
FfXlFCdJ6NQ850l8wZonhQ
mUi8T4_azCi1rxV1iTa7mA
Fo76GsZVYYlj3AzbHW4_dw
H2MUPwdFWJ-jlx2L7k07fQ
exIuIO0BPPW8hvbxLX0gRA
TgvCcPorIuTCQNh-M-mDYg
IPaLGmqETApQd4XUHPGh1g
DPtZ4A1m9bdJ-QS2rDECMg
rMKIn8WCBDGxKHm-E9ULvQ
uYwtf-iVZ6607eMDTVIZPQ
dx81sRk4M6W0lhigS9jF2Q
TZI0wjfOBd3RRWoRA2HF0A
4MmI-UVHQWJ1ac1zKc4f-w
b030I9J2-1jG6J9695UY_Q
hcwR_ZJGKe1kPZnHDDINQw
fK1ZuFyv_km0LnrgCKiEyw
wuacETfsQpEGvFFcCwQnDA
qiYR1-wH6lUHQ4M0pQc_kg
6mLx5aZjXu1tC8ixCernJg
8-mpx_kMlN1MPIhDTBrZYg
D8iVhmi5nttaHkljN8Ht2w
f0DwLI9_wu8eF2KRDf1uYA
TyeD6l0wCo4NwWijukjMeA
LWxlgYNAN3B

Y_fCsXGL11EYr1a0PM-AmQ
OniWp07EhuQaHPqvQK_5MA
nBtW70xBh9tr9gIzosct2w
WZIs2YPk5Wqnkyvt7d6AOw
la1dmIlrSFIR3TceMXHmZA
C2-WYbaRSE713J4BUPvyPA
SCo8faL_nz0tkFoGeGexLA
qaOi1eeDNQNS10nAYB8ncw
5sQ8jrdLxvIETvnZuOCZPA
P2TvQZTyIP51ySn9ZMi5VA
bIpb03KgUZnDhpMQs_2QfA
LYb2AxHFYHzIt9vbXU2G6g
O5oXb39SkNpJXzdDz9VBvQ
Jgy4F1AirbRBkosFKNSXpw
WZf8lU4lDIPcafd7OqxM3w
ojaI7qZsCDIj591L9jPJpA
wQtVea4Id1Xh7yPBS1sFXg
iiLAegIR0NHHQeur-Y6JvQ
GFMVH3YV9qAHCEXfV-0wmw
RDMh81BSKvZKOlEA51R_4g
e2Us-AfTFo60RgtxxJeSKg
5fYnj0rYCkgin8Fwa9Wf8A
QQ8kuF3s_Wy2V6s1i0Thpw
c7JAUBVtX3FkEmxXV9LL-w
Rz7M_8NFQpGTgWzZ2nezPA
pEpdjOo3UeD4i4K1iUSZlA
FXRQZLt_IV-BPjWbGxlLLA
tpz4tkyfzDh7Sre6CrQ_Ow
nNxJjs2gNXohqBRKpgWEWA
jnlBTr1DxqHK0pCFL4l0ww
YHAyReR35msUGbBX4I-9Qg
BH-dI2EMGbKzv8CBahy3Ew
AtSMEOFJRgAxpLxVZFKqsw
2jCLSzGyAq2b_quS3SFREw
DDvpbb8SvTzQgWsY5dt6XA
jcYa6zF0icj1A2XrsLWRFg
SXrHOzV7b9JMIBA86QemJQ
7JGxO9wMuXiiqoaHV4DU_Q
q2d8JhaYV8Qlls5vUwgTrQ
ZTYc99TsXS5v7dx617fgNw
bVoyg0amEESpwwW5I3v2bA
wrsOjD2OMoSEJtG_BJXBEA
wWcm0SNHVvBwTOAP8ix1mg
5DlmCG3PZ9E

t3S7fxoRGUriMLFZGLJCcQ
_okbOXWtrIzLed20dfmlYQ
uZyUiDgjfKZTr0m0WllGpg
RmR98Y9ofpRifB5CFx_ipg
ksUdFhFpZ2iKuQLSTPY0-w
-FyLoMtER_hHA1fsYL_UzQ
6dkmnnhoirOiYdIS6zpDtw
CgIXu2i5U1uhNxqmEsF_Pw
MRco_M-ZuqI1XaHCuxlSLA
ya6IMaj1f_hxQmDGotUegw
0JyCN8USeYVWr8I1OhOGwg
A4eJkhDUfWMsIzeE3XcGHQ
jkSFBGmlJZKXi-XRDOHP6w
qJY80RgPRfP7RoVvWOqjXg
QtQ0f9WYqhmwE-beip4DLw
2aJs9GBjk76Wf8m31OhoPA
aIt5v1mmJq4fP242CJitUw
bJlDeUT8x4XlYlQf9kGmKQ
inkhOt9S_pfR1dAxJEjcmA
OzwtGQLIHsfYOiLDai2UNg
il-nQRNPseti8pzZDyjvvw
g9dwX75XVdshspcd0cJbTw
eP8-TqLljH1hXJi_sK6K9w
FggFK8PYO8CFVk2acl9ReQ
pgRjbCkGlCLDdzUNRnPglQ
7_n2XOctBPKwuVWHWVNaxg
Asv83FpKGXDVR9zGxgRCQQ
Hv405tFTVcJ25f0NUS6ylQ
QBYMqFUZKv-q738bnG6W6g
nZ7eyAY2mw8KcEv18wv45g
egBVTfAUGsqJbAVFicYq9Q
nc0OV43MGT3PKciAnskfgQ
IneR7kP2d-W2HZK-7V_vaw
i4TQJEMQ5kkAsAUFAER2Hw
NfTROrpoA1TJ7C_6SqF84g
a_Uw9mBAPHfgO5YqmS6knQ
ke6iHozUI-v5hcad7wP96Q
7Z4GbXvIYEPsSxnJ2ZPbmA
In0yKn3sH6FcfqhTVEg2Aw
Wl8m6hTVNmQpZdujxNNaQA
3Fe48mI9nhK38ZjAQhFGKA
vHHxkwV0wFQDYeMGWOz1qg
Lp4oRaDfZTQmgzGUHpu2wg
kaeETTKAVt1

eBFOuwsDQAlGNpGWAixaRQ
IYuqBibv3NVdd12qsppXjQ
EEFEdzran2FizUpgMMp6cQ
uoyNLXfe2RRnCjlwl_ZkTw
DeasxeL8GO9kzAkKjQjhmQ
wH3X_zImLq-9ygGuuS-nUQ
ucAreTgEn69dyhTrhWJYaw
sPW6QxE10jJ8rWWg0n745Q
QUi9xaCiimLLhaifxWJA9A
JFtSDT-TtSz5rQeQOqKzMg
YNRqdPeRS85DvM8SpYDySQ
aVxzNcDKtNBlFDnk0A791g
T8JJXfRiXwPDZw5kOQVlxg
N3VW9EdTtHFOMYe_235Cfg
Stya3krjMxQdCxd_ZevLow
A_l2vBCB_P3cgiT_Y8EkaQ
6_clc3Ox_Q-CmF6YrcKKjw
rTNCWZUqPrais4A0qgvjuw
UuGJyyYxvRFGCPzqYdKJ_Q
opmanq-HeWZRFmFTdDUrFA
-FB1Hl66ek85xXdvHht1Sw
XEP5iRFUM9HmjC_JEhQXCw
LRd4sYKbC4WRzMX_GV_cFQ
6-9gpI52K6P83GfZep9uqA
FsYuYbuXEob6baAAL0WmBg
-jLdLy_vZLN6M18YJ53ryg
tQweQKyiSXk2_lxnJypUvw
47qAeIFSzqScM_Rrw6e4cw
v5au_xQ-bL8Kxuxo7HNS8w
u_NuHOLNkuAiFTWJFscQqQ
-GBWodLAVsFJ0rTO81dYZA
1WB_fMZNzieIucS2D07QPw
l_8K8fZzyWrqHv93bHBgaA
-5KFPIIDo25bA14r5SVUnQ
U-yfG4JWJI4nndiXF3OaTQ
8z7Lkoem01q-wbKdIIM8-Q
-vbDWqmhYaLsDM0gaB6Eig
fGEUdyrR4neRokmeRGVpsg
D1Qs3FMDQHHHWSXxxUFH2w
CWijTA3XCZVTTW4OdtuJOQ
woorEzy4RKd0Xlq2cIC4KQ
zAQeFe9qcpzJnA4whN2s4Q
2vKivTSUT0dJSxwxarB2vw
vwwSkXLovq7

r6tnVMif_qdV0YQs2LC2IA
oO6QX2h2c2j2LwSEzh_ykw
0NgZvK7vvE9-NOj0QLsNvA
Yczxl1Wpq4mik8YXRxt5FA
LwlcOYyilfbcA5mKy6eYlQ
jTflwi5j65J2tSa8C2OD3w
jzZdmYovDoHzrTP2Q9Xscg
W9Y7pOJxjLzu_14aNQ1ErA
n6vOLiswfJR5tzKApKRpMA
EqDLd3yUX7pB40cOTIiojQ
e_EQQxh1679pv_w0BspC8g
jblOfWFArUw2DRN7AEPVXw
dm8_eQmsyEMWKA1H1aPw0A
dIWjBztN_G89O098PObgSQ
0waR5Wzw9YiDWDSPt41ykQ
88aQXTEZottgZXCuiN1dXQ
JxDSdaEQCiBalgmEv06new
8gtMPN-cdwPqELx4dcgBpw
DoESGfysWXc2a5HR-ZTouw
OxjWFi8QK4WyW6II-M5wHA
iL7-M4_BVqcR6VLRLUWXcw
WOgwOQnvePVuFF64mG5_IA
9_VnKNkuMxBiVkxtSrg_xA
06fOuZ9QCgPHNbBMp5z1EQ
71FfZkSpamnNJiHNyt0tCg
sZ_iu_huISifmUPFjpMrJQ
v08g3BwIGByMTfcAvLBcow
jB3cVogCCX1p6xk9K-ltCw
y7ApvqF0S0pgIeruc1nauQ
PNjnxsdruclTbxJpRblHww
j6OtLE-8R3rmnEm6QB4tJA
58RIfXrNZ0OXB91rJQedlQ
tvIOfxDHlrQZb7rbltP3-A
6Wr_igHKAtYzFbDxF7JqMQ
3A7rYhyyLEFFmppwpAZU-Q
WUrqYJipYhymZuT6dPpKqQ
Da_1dRuGA6NBWk-TzDoTdw
p6U8XMG_KmRVn9ZAjd3HBQ
JhV-1Vo69MLO6AmZtrncIA
IzQsBeD4B1HfR-8KGU_KkQ
YQRbLBmxIbOWuXeC1JY-zA
SPfToHCY0OXIRcmtkDFbug
mhdJzr5Fwx0fVircPIcjgA
13dgAtF1T2P

s0JZqw-jlyUefE9tmiOjaQ
pLLiDE_mGn746kMKwlD3uQ
58r0ALuW3Xy_yOueLpyslw
5DA_qRPe9_buyMNORTxWnQ
saR_vafEl1qXisK4soQUFg
AHwW0Rgxbjewcas00Lp1Ig
Hqe9x_MZDHiaqAIiaIIvRw
nasTLlikLRNfbQUuX2QeJw
mCs1taXO-ujXPz4fyMSFSA
GpK1dyElqUrtJd7cQ9IUHg
AqgzE5fIFy4XBbqMs5r6bw
ytyUUyacL8zrYAIill2tRQ
9Ty0INrk0h38dqlSdtzhEw
KEEGmuyQiclKo-_8ZDcCUg
4o0fgRvpfVc0hwp6yAlHgw
F7zp1VXTt93DLWQ9jiS3fg
n0S5C2MdCAOfLLxbsFtKrQ
Rj9Si34DnnAh51RX34WrDw
FUWV-0EoUSRRDDOCVljOsw
GwwvPPzwedVqHWVJ0IodCg
MaGg9dAlh5U36o4fB5eDag
Rwlw9GWYb6-5dNixwFS4kg
leH01mWZVcyLAqkFW9zLWQ
gskQWvKiN3kR92qTJR-IUQ
BWc2M7C3YTf5b9F5nUzKjw
DWDbW7cr9xRxVM9NpB7LnQ
N5on17tuXq3b7mxLMgVv7A
WPV7DS7v6gkDBjfDgSu9-Q
rQvgGbTLpZ32lzq9Ya5bUA
0QiI6hkborsCpYTx0JXcjQ
dAQPtB4Q2aACaPKGrMjbJw
2nXdgBjYUZOdO8fnOo_0eA
86sPj0uijT_3qaTrVY5OpQ
vgAaSGvDXyaj7MtCat0sdQ
DeftpE6qlEuMIN4jcaBg6g
TlttJy6msDUzC-6b2C-daw
jAEj-DiDt50cpyZ5peFAlQ
3gikSWM6tGoMMOh0ozUvfA
N5IkWNX6nQJp7NT6ZzEZRw
w1UrSRM_PNtYu4sJEvb7AQ
RS8yXCiM9XECs7Sv_LA_bg
G4zkR23Diqf9SbBcXOcaPA
VE4aAqAInGfL5Axy0DWQxw
Mj7M1tM7FN4

UItr20fck9rnASwSvH1cCQ
wEGk2e_vMO_c5R1Pvo_Ctg
xEzlQst2hro4CSUrjWa6ww
-t3iQkvL7oMwS4y1elKbQw
2IVzFUZc4PKgZZtWSTn6Pw
4jkkzw0AM8zgEDq4ylv6aQ
cvx0JUGG81EKDyR1R4JjWg
_1H9SjTC97wfNUDb8vqCiQ
k2n9PhEyGyaGDADU-7LmVg
EQvk4g31hdOPisL1f5pfBw
H2clzvcBIHAAjDnWVfWNVQ
AGTE0q4KpZs17AFMxMNFpQ
2f07wJhb19HEMNQsHyR9ZQ
miqlmt9K_5EDaxZ3ubJBqQ
L_IR1s_OOpuk-5QpqZsxHg
16aXK8J2sKxKzakDkut3GA
SQE4XNVXnbS0bW4e6G2yZg
-X0fUMwsSfWgQ13obYNr_g
hZlSxrF3Zw7Jmiz8ozzByw
h4aZiVApiw3VVp7tdkfRzQ
kGGwbVdQykr3mqSLV-XWEw
ugvT0vNtxIIrKA3aK6RQgg
9fnY80pD_Y8J0zbqw4dqeQ
-oAoju-VJJmkXmCw7yluKA
TYiocgi95dAZ_egPXDqYRw
jzSAdouI26VrZklAbfg1mA
SP6-KPAHyAWQ8rVFuXwdWQ
0SCzsGAxkW6gs2QIQhgy6g
ASRQpxqYyOfnFM3uGASztQ
46IfoYpJbEaEKM1tiXSmOw
2aE1w4crc4l9ihjlzqfXqQ
Vfike-qbZK3kDFGqPe-Nqg
IfuOXp3pEmCnQK2UEnNZfw
S6BJIqy9zH8-fuwfw79GEg
Nbf_zO13aCxo3BdO4wtDxg
N_gX85bNuySlOgvEG87EhQ
0aUxwv5WDD03zGeX-zwzcQ
9EWYDu7gCCj4NV23wTuvEw
0FTkszlGT2kSA4IObDLYgA
-T7Lhwyn7L3gjWnYXZOPkQ
SxHORDomLYjhZEEUvS9R-w
p-EHMRXk3kVuFif6GbRV_w
nc_cOtixjt8bKJmSrXgJcg
MZI-u6IsVe5

ir-76ssaIKE5VyaMxEv3Fw
v1ewtJnzmKcn1jR6VWiflA
uzfcOOpkDdRSuNKnw_TN5Q
Mk7cOD5YbOQpnrE4wQ0DaA
4jB_0JXnmbd4vHvyPKTVnA
hMvGQ5x_5QUjSmpiRjau3Q
RrUqe6GzcWpTTK-ypN-ahA
0hBeLA5ygv_IH3uTIMsBxg
ftwmr9hVM1LFKnm7rrfQOg
cJ_VPkzAeIK0ws8UK4EVSA
kXJRWqDU4l8b87eMAhXSXw
E_9SH-ZseLHQnSv3ROBvGQ
2pnk96Jn0YACVktQxkvZmw
C09PGuBW-avM7Dp_Gv6BHg
VJGX3Kn-YEIHryxXUndUYw
xJ2KyhSPAa9iewQjGgF2MQ
i415qPgZAfu1kYpIf_MeFw
DmL4ILcDhFi_x_Eqqb1LlA
rhgpBXG4jr5S9NLeW4CVzA
HQqcEj6YHhbq3zRo_qIWqA
olvfFlTTT4HF3EKWVYj8Dw
ESLGREROq5bDzmabLChnLw
1xtq7JA1ikRW6xvWNsrd1g
pk-F0k4BUki983c7mjvrhQ
hFnNg1JukEnMftEtIoRZbA
xEuCgcvlu0DxXVzyrjt5Ow
cALBmFyiBEDDjwemVtF5ig
9JpXH3Kq9RA6gsKqRnej9Q
SvyVz9m9Vuvc5_ugpxumjQ
IhwQ2J5UHhe7j1u9vkp8ng
P1nyfbcV9Gzo5C_LQoe7vQ
xOHfQ2lLjv0vl6yDeSwwXw
eFRhfDKAIcyjTyCF6DgdUg
cZ01DQ-yDBf8AUHuRFq4Og
LqsWmr_I6qtiaAYRTPA8sQ
qIvZ8EJhp1NI7kM-VI2Q_w
mF6E6R3JQzJCMfLYHzJ6Dw
2V4d4HC-kA6uT0Ng1aVlOQ
PjP-7FWLDgyRO64CXQJAWw
YzYjuszvf_v0ktfpercJQg
1p8yVv_RpbMsx2usFtmbCA
jk2H9BK880dxr7ddFXR6PA
7aJGX7-Cth-09B71MFx7PA
pfKguEKOG6-

zjLpKpU6-g441f9OLEYV5A
xdyXD4FTpyrz7i9QdhBO5w
cNRInaBmfFvO-0H1w5AXFw
FPL8Rmi3Fr9hVVunZjXs8A
4hXC8YftALnR55TtFiDzlw
-3iqW89-7SFvRFWb2Cir6Q
-kemQaZtZZR1SEOX98D-TA
724RcaGow5QhrOcpf9tWXA
E4iGaIin1pu1zL36-OSyFw
sHfBkZiRJB95x2nALGmgtw
G3heTDW4aqEsRm2pbQi00A
_EBe4KAlzsloTA8xyMzHHA
VTgBQuCt8uuMtpwiq2vqTg
YLVVNUjaD9wfnCefW8WOVw
VWWbAVPz45aW2VStDVYpPQ
Dm5FfGM4cUPev0x_7IxU4g
el_MelUwCgG0Ejnn4xzrAA
7epOlncRqx57_pM_VSpMYA
vXLB5LcBNwgddgviOFgCvA
BM20UMQ89fplRUKltJY15A
f1a0sQIpJ6x9rYJSFqNzyw
8XAoGp4zVR57ltQ60_J8Ag
KmeBb_AAFNlO8z2m8CorsQ
7kc4GfyU25C5QDOzj4XOCQ
m1qWF1M8trKkKomj8Ueq7A
KfZHF623tyY3ukWbHHiB5Q
LGfVqzowD6cCqZn7IuwR4g
PLT7OcUr3IUSlCfWK16alQ
ThzzR9BJcnocD3TYJb1Aww
hLOv6jMZ-mO_q6WTVW5i3A
oq9IDkpF6YYhN6GQDcoXeA
eto4dAobmxBDa5bS7EVlBw
P0nNw-xQ6eOM41FO-iCuXA
9qttDbOeZzmtgXwjsd6_yw
G_ytUNHY8Ac5Rcgz5iYalQ
UFaqH6EHRi7CL3X069Kgcw
HjU2EHgTnJSxRqVvXu53OA
ovB8U_vsgAU5Emw_krzzqg
cDnj244yCT-Fv7t_QGxGQA
Z3stw-lKNZWVBL_VHCAdRQ
V4ys_CcCsbvJefK7GQ4BkA
NFEEkzLptfK7Q4M6poHMgQ
iQJ7hBdiw7odTeqwBmP9Aw
ERAq1TP-J0E

In [396]:
print(reviews_dict['8jYsEl63Lm3ckUEvYkW4kQ'])
print()
print(reviews_dict['AHG4DZ2GYjJXcg7LyvPMhQ'])
print()
print(reviews_dict['JVITPvg5vCJ6AUeY8yQC6g'])
print()
print(reviews_dict['2jlf6NA5ZetNOO4jpVLWqg'])
print()
print(reviews_dict['99end7AF6NiJsn2AUBRzdw'])
print()
print(reviews_dict['pVQql9djXWWXO6NyZDf9HQ'])
print()
print(reviews_dict['z9jmAT4uV72TmMeAu0Ixng'])

I end up here all the time with family or friends come to visit and want a taste of local Phoenix flare - but I admit I have a tendency to lean towards gastropubs. I love their beer, and they have great food options, even for vegetarians! The tables outside are fantastic in the fall and the setup inside is great! One of my favorite go-to's :)

I visited this brewery last week, while we were celebrating my brother's 21st birthday in AZ. I was looking for a bar and came across this place on Yelp so we decided to give it a try since it was only 30 minutes away from our hotel.We got there around 8 on a Sunday and easily found parking. The place was pretty nice and we chose to sit in their patio. Within a few minutes, came over waitress Aileen and as I was telling her about what we're thinking of ordering and how it's my brothers birthday, she paused me and told me to choose and that she'll be back! I was very offended but didn't make a big deal out of it since it was my brothers bday. The 

In [378]:
with open('../data/yelp_businesses.pickle', 'rb') as pickleReader:
    business_dict = pickle.load(pickleReader)

In [389]:
with open('../data/yelp_businesses.pickle', 'wb') as pickleWriter:
    pickle.dump(business_dict, pickleWriter, protocol=2)

In [397]:
print('Businesses:',len(business_dict))
print('Reviews:', len(reviews_dict))

Businesses: 74515
Reviews: 14187


In [385]:
nbr_key= 0
while nbr_key < 2300:
    business_dict[business_ids[nbr_key]]['reviews_pulled'] = True
    nbr_key += 1

In [387]:
nbr_key = 0
for bid in business_ids:
    if 'reviews_pulled' in business_dict[bid].keys():
        nbr_key += 1
        
print('Key:', nbr_key)

Key: 2300


In [400]:
soup = BeautifulSoup(r.content, "lxml")
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html>
<head>
<title>Yelp Captcha</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="no-cache" http-equiv="Cache-Control"/>
<meta content="0" http-equiv="Expires"/>
<link href="/favicon.ico" rel="shortcut icon" type="image/ico"/>
<style type="text/css">
    html, body, div, span, applet, object, iframe,
    h1, h2, h3, h4, h5, h6, p, blockquote, pre,
    a, abbr, acronym, address, big, cite, code,
    del, dfn, em, font, img, ins, kbd, q, s, samp,
    small, strike, strong, sub, sup, tt, var,
    dl, dt, dd, ol, ul, li,
    fieldset, form, label, legend,
    table, caption, tbody, tfoot, thead, tr, th, td {
        margin: 0;
        padding: 0;
        border: 0;
        outline: 0;
        font-weight: inherit;
        font-style: inherit;
        font-size: 100%;
        font-family: inherit;
        vertic

In [407]:
review = 'Op_IdFSIbwC1gKU5ueABfA'
div = soup.find("div", {"data-review-id": review})

In [412]:
div.find('div', {'class': 'i-stars'})['title']

'5.0 star rating'

In [406]:
review_ids_list

['Op_IdFSIbwC1gKU5ueABfA',
 'pdn1MWbN7C1aG009S1SxSQ',
 'xesARNaf-mk2C9r2KAtXoA',
 'YWvTYpMzwD8ueR7CqDoW-Q',
 'y7sZu6FxxjajmagC23foHQ',
 'A7Ym-kLZLnWgodVma3auig',
 'aO5ZZ5NAlraCBePAt9qJRQ',
 'X0PJN9rRDcK5329qfKAsRQ',
 'R158mvv5UISq182quGiZeA',
 'mPe7Zxg6NYJrersiRnVoFw',
 'jJKmHVOIibg4m5v1Qyj1ew',
 't_5sm7K1dTb_q-TIEWr4gA',
 'Wv3PMFMHz7s4XEXria92vw',
 'PlOIQVOiK83GaHmJrm3fjQ',
 'nI80758kF7iJzwGOeFraeQ',
 'zMv0c8O_nIiw6Pzl9bRB2g',
 'uooDHBypzlH_CWNSi37aMQ',
 '57zfMbSkSyYO-_d11ycX-A',
 'fAIRFewsPXB0NDK_6GHyzA',
 'B91zEQUOTlsnYc0ymDtJHA',
 'ytsnT8MroN3XPPjJHaLgLw',
 'NyfA4uOgFXXaDrms8IxGhg',
 'BpwPKasr5A1jaMbARO7IOA',
 'kFJnJzYj0Vx7C1rjcVTdXw']